Karolina Filipiuk: kfilipiuk@student.agh.edu.pl

Radosław Mikołajczyk: rmikolajczyk@student.agh.edu.pl

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from keras.datasets import mnist
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from keras import models, layers
from keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


## [deskew]
SZ=28
affine_flags = cv2.WARP_INVERSE_MAP|cv2.INTER_LINEAR

def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img
## [deskew]

def showOpencvImage(image, isGray=False):
    fig = plt.figure(figsize=(6, 6))
    plt.imshow(image, cmap = 'gray')
    plt.show()

def openCVHOG(im):
    winSize = (20,20)
    blockSize = (10,10)
    blockStride = (5,5)
    cellSize = (10,10)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    signedGradients = True

    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradients)
    descriptor = np.ravel(hog.compute(im))
    
    return descriptor

Wczytanie oraz wstępne przetwarzanie danych

In [3]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Wektoryzowane surowe dane
train_raw = train_images.reshape(len(train_images), 28 * 28)
test_raw = test_images.reshape(len(test_images), 28 * 28)

# Dane zmieszane
train_shuffle_data = np.random.permutation(train_raw)
test_shuffle_data = np.random.permutation(test_raw)


# Dane wyrównane
train_deskewed = np.float32([deskew(im) for im in train_raw])
test_deskewed = np.float32([deskew(im) for im in test_raw])
train_deskewed = np.asarray(train_deskewed).reshape(-1,28*28)
test_deskewed = np.asarray(test_deskewed).reshape(-1,28*28)


# Dane hog descriptor
hogdata_train = np.float32([openCVHOG(im) for im in train_images]).reshape(-1,81)
hogdata_test = np.float32([openCVHOG(im) for im in test_images]).reshape(-1,81)
hogdata_train_deskewed = np.float32([openCVHOG(deskew(im)) for im in train_images]).reshape(-1,81)
hogdata_test_deskewed = np.float32([openCVHOG(deskew(im)) for im in test_images]).reshape(-1,81)


# Dane do wyszukiwania w siatce i przeszukiwania siatki z walidacją krzyżową
hogdata_train_deskewed_short = hogdata_train_deskewed[:600]
train_labels_short = train_labels[:600]
hogdata_train_short = hogdata_train[:600]

# przygotowanie modelu dla sieci neuronowej
def create_model():
  model = models.Sequential()
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dense(10, activation='softmax'))

  model.compile(optimizer='rmsprop',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
  return model

11493376/11490434 [==============================] - 0s 0us/step


# **Zadanie 2**

Porównanie wyników działania klasyfikatorów dla poniższych przypadków:

1.   Obrazy oryginalne->Deskew->HOG->Klasyfikator (SVM, Las losowy, Sieć)
2.   Obrazy oryginalne->HOG->Klasyfikator (SVM, Las losowy, Sieć)



**SVM**

In [ ]:
#Walidacja krzyżowa z wykorzystaniem GridsearchCV

deskew_parameters = {'kernel':('linear', 'rbf'), 'C': np.linspace(start = 0.001, stop = 2, num = 200)}
deskew_model = GridSearchCV(svm.SVC(), deskew_parameters, scoring='accuracy', cv=5)
deskew_model.fit(hogdata_train_deskewed_short, train_labels[:600])
deskew_params = deskew_model.best_params_
deskew_svc_best_estimator = deskew_model.best_estimator_
print("Chosing SVM params for deskewed data: ", deskew_params)
print("Best SVM estimator for deskewed data: ", deskew_svc_best_estimator)

print('\n\n')

parameters = {'kernel':('linear', 'rbf'), 'C': np.linspace(start = 0.001, stop = 2, num = 200)}
model = GridSearchCV(svm.SVC(), parameters, scoring='accuracy', cv=5)
model.fit(hogdata_train_short, train_labels[:600])
params = model.best_params_
svc_best_estimator = model.best_estimator_
print("Chosing SVM params for non-deskewed data: ", params)
print("Best SVM estimator for non-deskewed data: ", svc_best_estimator)


pred_labels_deskewed = deskew_model.predict(hogdata_test_deskewed)
pred_labels = model.predict(hogdata_test)

print("SVM Accuracy for deskewed images: {}".format(accuracy_score(test_labels, pred_labels_deskewed)))
print("SVM Accuracy for non-deskewed images: {}\n".format(accuracy_score(test_labels, pred_labels)))

print("SVM Confusion matrix for deskewed images:\n{}".format(confusion_matrix(test_labels, pred_labels_deskewed)))
print("SVM Classification report for deskewed images:\n{}\n\n".format(classification_report(test_labels, pred_labels_deskewed)))

print("SVM Confusion matrix for non-deskewed images:\n{}".format(confusion_matrix(test_labels, pred_labels)))
print("SVM Classification report for non-deskewed images:\n{}\n\n".format(classification_report(test_labels, pred_labels)))

Chosing params for deskewed data:  {'C': 0.714211055276382, 'kernel': 'linear'}
Best estimator for deskewed data:  SVC(C=0.714211055276382, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)



Chosing params for deskewed data:  {'C': 0.864889447236181, 'kernel': 'linear'}
Best estimator for deskewed data:  SVC(C=0.864889447236181, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
Accuracy for deskewed images: 0.9374
Accuracy for non-deskewed images: 0.9151

Confusion matrix for deskewed images:
[[ 950    3    3    0    3    6    2    3    0   10]
 [   1 1124    2    0    0    0    1    6    1    0]
 [   2    

Na podstawie powyższych rezultatów można zauważyc, że klasyfikator SVM działa lepiej dla obrazów z wyrównaniem. 

*   Accuracy dla obrazów z wyrównaniem: 0.9374
*   Accuracy dla obrazów bez wyrównania: 0.9151



**Las losowy**

In [ ]:
parameters_for_cross_validation = {'max_depth':[5, 10, 15], 'n_estimators': [80,100,120], 'max_features': [30, 45, 60]}
deskew_model = GridSearchCV(RandomForestClassifier(), parameters_for_cross_validation, cv=5, scoring='accuracy')
deskew_model.fit(hogdata_train_deskewed_short, train_labels[:600])
deskew_params = deskew_model.best_params_
deskew_rfc_best_estimator = deskew_model.best_estimator_
print("Chosing RandomForestClassifier params for deskewed data: ", deskew_params)
print("Best RandomForestClassifier estimator for deskewed data: ", deskew_rfc_best_estimator)

print('\n\n')

model = GridSearchCV(RandomForestClassifier(), parameters_for_cross_validation, cv=5, scoring='accuracy')
model.fit(hogdata_train_short, train_labels[:600])
params = model.best_params_
rfc_best_estimator = model.best_estimator_
print("Chosing RandomForestClassifier params for non-deskewed data: ", params)
print("Best RandomForestClassifier estimator for non-deskewed data: ", rfc_best_estimator)

pred_labels_deskewed = deskew_model.predict(hogdata_test_deskewed)
pred_labels = model.predict(hogdata_test)

print("RFC Accuracy for deskewed images: {}".format(accuracy_score(test_labels, pred_labels_deskewed)))
print("RFC Accuracy for non-deskewed images: {}\n".format(accuracy_score(test_labels, pred_labels)))

print("RFC Confusion matrix for deskewed images:\n{}".format(confusion_matrix(test_labels, pred_labels_deskewed)))
print("RFC Classification report for deskewed images:\n{}\n\n".format(classification_report(test_labels, pred_labels_deskewed)))

print("RFC Confusion matrix for non-deskewed images:\n{}".format(confusion_matrix(test_labels, pred_labels)))
print("RFC Classification report for non-deskewed images:\n{}\n\n".format(classification_report(test_labels, pred_labels)))

Chosing RandomForestClassifier params for deskewed data:  {'max_depth': 10, 'max_features': 30, 'n_estimators': 80}
Best RandomForestClassifier estimator for deskewed data:  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features=30,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=80,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)



Chosing RandomForestClassifier params for non-deskewed data:  {'max_depth': 10, 'max_features': 30, 'n_estimators': 120}
Best RandomForestClassifier estimator for non-deskewed data:  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       crit

Na podstawie powyższych rezultatów można zauważyc, że klasyfikator RFC działa lepiej dla obrazów z wyrównaniem. 

*   Accuracy dla obrazów z wyrównaniem: 0.9075
*   Accuracy dla obrazów bez wyrównania: 0.8745

**Sieć neuronowa**

In [ ]:
nn_train_deskewed = np.array(hogdata_train_deskewed).reshape((60000, 81))
nn_train_deskewed = nn_train_deskewed.astype('float32') / 255

nn_test_deskewed = np.array(hogdata_test_deskewed).reshape((10000, 81))
nn_test_deskewed = nn_test_deskewed.astype('float32') / 255

nn_train_non_deskewed = np.array(hogdata_train).reshape((60000, 81))
nn_train_non_deskewed = nn_train_non_deskewed.astype('float32') / 255

nn_test_non_deskewed = np.array(hogdata_test).reshape((10000, 81))
nn_test_non_deskewed = nn_test_non_deskewed.astype('float32') / 255

encoded_train_labels = to_categorical(train_labels)
encoded_test_labels = to_categorical(test_labels)

parameters_for_cross_validation = {'epochs':[1,6,12], 'batch_size':[32,64,128]}
deskew_network = KerasClassifier(create_model)
deskew_model = GridSearchCV(estimator=deskew_network, param_grid=parameters_for_cross_validation, cv=5)
deskew_model.fit(nn_train_deskewed, encoded_train_labels)
deskew_params = deskew_model.best_params_
print("Chosing Neural Network params for deskewed data: ", deskew_params)

network = KerasClassifier(build_fn=create_model, verbose=0)
model = GridSearchCV(estimator=network, param_grid=parameters_for_cross_validation, cv=5)
model.fit(nn_train_non_deskewed, encoded_train_labels)
params = model.best_params_
print("Chosing Neural Network params for non-deskewed data: ", params)

pred_labels_deskewed = deskew_model.predict(nn_test_deskewed)
pred_labels_non_deskewed = model.predict(nn_test_non_deskewed)


print("NN Accuracy for deskewed data: {}".format(accuracy_score(test_labels, pred_labels_deskewed)))
print("NN Accuracy for non-deskewed data: {}\n\n".format(accuracy_score(test_labels, pred_labels_non_deskewed)))

print("NN Confusion matrix for deskewed images:\n{}".format(confusion_matrix(test_labels, pred_labels_deskewed)))
print("NN Classification report for deskewed images:\n{}\n\n".format(classification_report(test_labels, pred_labels_deskewed)))

print("NN Confusion matrix for non-deskewed images:\n{}".format(confusion_matrix(test_labels, pred_labels_non_deskewed)))
print("NN Classification report for non-deskewed images:\n{}\n\n".format(classification_report(test_labels, pred_labels_non_deskewed)))

375/375 [==============================] - 1s 1ms/step - loss: 1.4079 - accuracy: 0.7008
Epoch 1/6
1500/1500 [==============================] - 3s 2ms/step - loss: 2.1490 - accuracy: 0.2929
Epoch 2/6
1500/1500 [==============================] - 2s 2ms/step - loss: 1.2581 - accuracy: 0.7124
Epoch 3/6
1500/1500 [==============================] - 2s 2ms/step - loss: 0.7631 - accuracy: 0.8161
Epoch 4/6
1500/1500 [==============================] - 2s 2ms/step - loss: 0.5349 - accuracy: 0.8597
Epoch 5/6
1500/1500 [==============================] - 2s 2ms/step - loss: 0.4224 - accuracy: 0.8834
Epoch 6/6
375/375 [==============================] - 1s 1ms/step - loss: 0.3118 - accuracy: 0.9132
Epoch 1/6
1500/1500 [==============================] - 3s 2ms/step - loss: 2.1384 - accuracy: 0.3009
Epoch 2/6
1500/1500 [==============================] - 2s 2ms/step - loss: 1.2468 - accuracy: 0.7103
Epoch 3/6
1500/1500 [==============================] - 2s 2ms/step - loss: 0.7385 - accuracy: 0.8170
Epoc

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


NN Accuracy for deskewed data: 0.9516
NN Accuracy for non-deskewed data: 0.9362


NN Confusion matrix for deskewed images:
[[ 954    1    3    0    2    3    9    1    1    6]
 [   2 1117    2    0    6    0    4    2    2    0]
 [   0    2  987    2    3    0    2   20   15    1]
 [   0    1   18  951    1    9    0   12   15    3]
 [   2    1    1    0  967    1    8    0    0    2]
 [   4    0    1    3    6  841    4    1   22   10]
 [   6    3    0    0   13    3  930    0    3    0]
 [   8    4   45   20    5    0    0  922    7   17]
 [   1    0    3    7   12    2    6    4  924   15]
 [   9    1    3    1   12    7    8    9   36  923]]
NN Classification report for deskewed images:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       980
           1       0.99      0.98      0.99      1135
           2       0.93      0.96      0.94      1032
           3       0.97      0.94      0.95      1010
           4       0.94      

Na podstawie powyższych rezultatów można zauważyc, że Sieć neuronowa działa lepiej dla obrazów z wyrównaniem. 

*   Accuracy dla obrazów z wyrównaniem: 0.9516
*   Accuracy dla obrazów bez wyrównania: 0.9362



Po porównaniu obu scenariuszy z treści zadania, możemy stwierdzić, że wyrównanie obrazów za pomocą funkcji *deskew* przed zastosowaniem deskryptora HOG zwiększa wydajność każdego z testowanych modeli. Dzięki zastosowaniu funkcji *deskew* elementy treningowe były do siebie zbliżone, więc klasyfikator był w stanie łatwiej znaleźć wspólne cechy.

# **Zadanie 3**

Porównanie wyników działania klasyfikatorów dla poniższych przypadków:

1.   Obrazy oryginalne -> Deskew -> HOG -> Klasyfikator (SVM, Las losowy, Sieć)
2.   Obrazy oryginalne -> Deskew -> ReshapeTo1D -> Klasyfikator (SVM, Las losowy, Sieć)


**SVM**

In [34]:
parameters = {'kernel':('linear', 'rbf'), 'C': np.linspace(start = 0.001, stop = 2, num = 200)}

hog_model = GridSearchCV(svm.SVC(), parameters, scoring='accuracy', cv=5)
hog_model.fit(hogdata_train_deskewed[:300], train_labels[:300])
hog_params = hog_model.best_params_
print("Chosing SVM params with HOG data: ", hog_params, '\n')

reshape_train = train_images[:300].reshape(len(train_images[:300]), 28*28)
reshape_train = reshape_train.astype('float32')/255

reshape_test = test_images[:300].reshape(len(test_images[:300]), 28*28)
reshape_test = reshape_test.astype('float32')/255

reshape_model = GridSearchCV(svm.SVC(), parameters, scoring='accuracy', cv=5)
reshape_model.fit(reshape_train, train_labels[:300])
reshape_params = reshape_model.best_params_
print("Chosing SVM params without HOG data: ", reshape_params, '\n')

pred_labels_HOG = hog_model.predict(hogdata_test_deskewed[:300])
pred_labels = reshape_model.predict(reshape_test)

print("SVM Accuracy with HOG images: {}".format(accuracy_score(test_labels[:300], pred_labels_HOG)))
print("SVM Accuracy without HOG images: {}\n".format(accuracy_score(test_labels[:300], pred_labels)))

print("SVM Confusion matrix with HOG images:\n{}".format(confusion_matrix(test_labels[:300], pred_labels_HOG)))
print("SVM Classification report with HOG images:\n{}\n\n".format(classification_report(test_labels[:300], pred_labels_HOG)))

print("SVM Confusion matrix without HOG images:\n{}".format(confusion_matrix(test_labels[:300], pred_labels)))
print("SVM Classification report without HOG images:\n{}\n\n".format(classification_report(test_labels[:300], pred_labels)))

Chosing SVM params with HOG data:  {'C': 0.5635326633165829, 'kernel': 'linear'} 

Chosing SVM params without HOG data:  {'C': 1.7086884422110553, 'kernel': 'rbf'} 

SVM Accuracy with HOG images: 0.9033333333333333
SVM Accuracy without HOG images: 0.81

SVM Confusion matrix with HOG images:
[[24  0  0  0  0  0  0  0  0  0]
 [ 0 41  0  0  0  0  0  0  0  0]
 [ 0  1 30  0  0  0  0  0  0  1]
 [ 0  0  1 22  0  0  0  0  1  0]
 [ 0  0  0  0 34  0  3  0  0  0]
 [ 0  0  0  0  0 27  0  0  0  2]
 [ 0  0  0  0  0  0 24  0  0  0]
 [ 0  1  6  2  0  0  0 24  0  1]
 [ 0  1  0  1  1  0  0  0 13  5]
 [ 0  0  0  0  0  0  1  0  1 32]]
SVM Classification report with HOG images:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        24
           1       0.93      1.00      0.96        41
           2       0.81      0.94      0.87        32
           3       0.88      0.92      0.90        24
           4       0.97      0.92      0.94        37
         

Na podstawie powyższych rezultatów można zauważyć, że klasyfikator SVM działa lepiej dla obrazów z wykorzystaniem deskryptora HOG.

*   Accuracy dla obrazów z wykorzystaniem deskryptora HOG: 0.9033
*   Accuracy dla obrazów bez wykorzystania deskryptora HOG: 0.81
    



**Las losowy**

In [37]:
parameters_for_cross_validation = {'max_depth':[5, 10, 15], 'n_estimators': [80,100,120], 'max_features': [30, 45, 60]}
hog_model = GridSearchCV(RandomForestClassifier(), parameters_for_cross_validation, cv=5, scoring='accuracy')
hog_model.fit(hogdata_train_deskewed[:300], train_labels[:300])
hog_params = hog_model.best_params_
print("Chosing RandomForestClassifier params with HOG data: ", hog_params, '\n')

reshape_train = train_images[:300].reshape(len(train_images[:300]), 28*28)
reshape_train = reshape_train.astype('float32')/255

reshape_test = test_images[:300].reshape(len(test_images[:300]), 28*28)
reshape_test = reshape_test.astype('float32')/255

reshape_model = GridSearchCV(RandomForestClassifier(), parameters_for_cross_validation, cv=5, scoring='accuracy')
reshape_model.fit(reshape_train, train_labels[:300])
params = reshape_model.best_params_
print("Chosing RandomForestClassifier params without HOG data: ", params, '\n')

pred_labels_HOG = hog_model.predict(hogdata_test_deskewed[:300])
pred_labels = reshape_model.predict(reshape_test)

print("RFC Accuracy with HOG images: {}".format(accuracy_score(test_labels[:300], pred_labels_HOG)))
print("RFC Accuracy without HOG images: {}\n".format(accuracy_score(test_labels[:300], pred_labels)))

print("RFC Confusion matrix with HOG images:\n{}".format(confusion_matrix(test_labels[:300], pred_labels_HOG)))
print("RFC Classification report with HOG images:\n{}\n\n".format(classification_report(test_labels[:300], pred_labels_HOG)))

print("RFC Confusion matrix without HOG images:\n{}".format(confusion_matrix(test_labels[:300], pred_labels)))
print("RFC Classification report without HOG images:\n{}\n\n".format(classification_report(test_labels[:300], pred_labels)))

Chosing RandomForestClassifier params with HOG data:  {'max_depth': 10, 'max_features': 30, 'n_estimators': 80} 

Chosing RandomForestClassifier params without HOG data:  {'max_depth': 15, 'max_features': 45, 'n_estimators': 100} 

RFC Accuracy with HOG images: 0.9066666666666666
RFC Accuracy without HOG images: 0.6633333333333333

RFC Confusion matrix with HOG images:
[[24  0  0  0  0  0  0  0  0  0]
 [ 0 41  0  0  0  0  0  0  0  0]
 [ 0  2 29  1  0  0  0  0  0  0]
 [ 0  0  1 23  0  0  0  0  0  0]
 [ 0  0  0  0 36  0  1  0  0  0]
 [ 1  0  0  0  0 27  0  0  0  1]
 [ 1  0  0  0  0  0 23  0  0  0]
 [ 0  0  2  4  0  0  0 27  0  1]
 [ 0  1  0  1  1  1  0  0 14  3]
 [ 2  0  0  0  2  0  1  0  1 28]]
RFC Classification report with HOG images:
              precision    recall  f1-score   support

           0       0.86      1.00      0.92        24
           1       0.93      1.00      0.96        41
           2       0.91      0.91      0.91        32
           3       0.79      0.96    

Na podstawie powyższych rezultatów można zauważyć, że klasyfikator RFC działa lepiej dla obrazów z wykorzystaniem deskryptora HOG.

*   Accuracy dla obrazów z wykorzystaniem deskryptora HOG: 0.9066
*   Accuracy dla obrazów bez wykorzystania deskryptora HOG: 0.6633


**Sieć neuronowa**

In [11]:
reshape_train = train_images.reshape((60000, 28*28))
reshape_train = reshape_train.astype('float32')/255

reshape_test = test_images.reshape((10000, 28*28))
reshape_test = reshape_test.astype('float32')/255

nn_train_hog = np.array(hogdata_train_deskewed).reshape((60000, 81))
nn_train_hog = nn_train_hog.astype('float32') / 255

nn_test_hog = np.array(hogdata_test_deskewed).reshape((10000, 81))
nn_test_hog = nn_test_hog.astype('float32') / 255


encoded_train_labels = to_categorical(train_labels)
encoded_test_labels = to_categorical(test_labels)

parameters_for_cross_validation = {'epochs':[1,6,12], 'batch_size':[32,64,128]}
hog_network = KerasClassifier(create_model)
hog_model = GridSearchCV(estimator=hog_network, param_grid=parameters_for_cross_validation, cv=5)
hog_model.fit(nn_train_hog, encoded_train_labels)
deskew_params = hog_model.best_params_
print("Chosing Neural Network params with HOG data: ", deskew_params)

network = KerasClassifier(build_fn=create_model)
reshape_model = GridSearchCV(estimator=network, param_grid=parameters_for_cross_validation, cv=5)
reshape_model.fit(reshape_train, encoded_train_labels)
params = reshape_model.best_params_
print("Chosing Neural Network params without HOG data: ", params)


pred_labels_HOG = hog_model.predict(nn_test_hog)
pred_labels = reshape_model.predict(reshape_test)

print("NN Accuracy with HOG images: {}".format(accuracy_score(test_labels, pred_labels_HOG)))
print("NN Accuracy without HOG images: {}\n".format(accuracy_score(test_labels, pred_labels)))

print("NN Confusion matrix with HOG images:\n{}".format(confusion_matrix(test_labels, pred_labels_HOG)))
print("NN Classification report with HOG images:\n{}\n\n".format(classification_report(test_labels, pred_labels_HOG)))

print("NN Confusion matrix without HOG images:\n{}".format(confusion_matrix(test_labels, pred_labels)))
print("NN Classification report without HOG images:\n{}\n\n".format(classification_report(test_labels, pred_labels)))

375/375 [==============================] - 1s 1ms/step - loss: 1.4936 - accuracy: 0.5997
Epoch 1/6
1500/1500 [==============================] - 3s 2ms/step - loss: 2.1372 - accuracy: 0.3071
Epoch 2/6
1500/1500 [==============================] - 3s 2ms/step - loss: 1.2500 - accuracy: 0.7131
Epoch 3/6
1500/1500 [==============================] - 3s 2ms/step - loss: 0.7538 - accuracy: 0.8168
Epoch 4/6
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5240 - accuracy: 0.8612
Epoch 5/6
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4077 - accuracy: 0.8860
Epoch 6/6
375/375 [==============================] - 1s 1ms/step - loss: 0.2935 - accuracy: 0.9198
Epoch 1/6
1500/1500 [==============================] - 3s 2ms/step - loss: 2.1419 - accuracy: 0.2939
Epoch 2/6
1500/1500 [==============================] - 3s 2ms/step - loss: 1.2751 - accuracy: 0.7036
Epoch 3/6
1500/1500 [==============================] - 3s 2ms/step - loss: 0.7719 - accuracy: 0.8171
Epoc

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


NN Accuracy with HOG images: 0.9545
NN Accuracy without HOG images: 0.979

NN Confusion matrix with HOG images:
[[ 960    1    3    0    1    3    4    2    1    5]
 [   1 1120    1    1    2    0    2    6    1    1]
 [   1    2  983   12    0    0    1   27    4    2]
 [   0    0   13  976    0    6    0   12    2    1]
 [   2    3    4    0  949    1    8    1    2   12]
 [   5    0    1    6    3  860    2    1    7    7]
 [  13    3    0    0   12    3  924    0    3    0]
 [   4    2   38   27    0    0    0  946    3    8]
 [   2    0    6   14    9    5    1    4  901   32]
 [  10    1    6    1    4   11    4   23   23  926]]
NN Classification report with HOG images:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       980
           1       0.99      0.99      0.99      1135
           2       0.93      0.95      0.94      1032
           3       0.94      0.97      0.95      1010
           4       0.97      0.97      0.97 

Na podstawie powyższych rezultatów można zauważyć, że Sieć neuronowa działa nieco lepiej dla obrazów bez wykorzystania deskryptora HOG.

*   Accuracy dla obrazów z wykorzystaniem deskryptora HOG: 0.9545
*   Accuracy dla obrazów bez wykorzystania deskryptora HOG: 0.979


Accuracy jest znacznie lepsze w przypadku klasyfikatorów SVM oraz Lasu losowego, które wykorzystują deskryptor HOG. Wykorzystanie deskryptora HOG pozwala zredukować ilość informacji tylko do cech niezbędnych do rozpoznania obrazu, co redukuje szansę na błędną klasyfikację obrazu, kiedy obrazy niewiele się różnią. W przypadku Sieci neuronowych wynik jest nieco lepszy dla danych surowych.

# **Zadanie 4**

Porównanie wyników działania klasyfikatorów dla poniższych przypadków:

1.   Obrazy oryginalne->Deskew-> ReshapeTo1D -> Klasyfikator (SVM, Las losowy, Sieć)
2.   Obrazy oryginalne->Deskew-> ReshapeTo1D -> Shuffle -> Klasyfikator (SVM, Las losowy, Sieć)

**SVM**

In [9]:
parameters = {'kernel':('linear', 'rbf'), 'C': np.linspace(start = 0.001, stop = 2, num = 100)}

non_shuffle_model = GridSearchCV(svm.SVC(), parameters, scoring='accuracy', cv=5)
non_shuffle_model.fit(train_raw[:600], train_labels[:600])
non_shuffle_params = non_shuffle_model.best_params_
print("Chosing SVM params without shuffled data: ", non_shuffle_params, '\n')


shuffle_model = GridSearchCV(svm.SVC(), parameters, scoring='accuracy', cv=5)
shuffle_model.fit(train_shuffle_data[:600], train_labels[:600])
shuffle_params = shuffle_model.best_params_
print("Chosing SVM params with shuffled data: ", shuffle_params, '\n')

pred_labels_non_shuffled = non_shuffle_model.predict(np.asarray(test_raw[:600]))
pred_labels = shuffle_model.predict(test_shuffle_data[:600])

print("SVM Accuracy without shuffled data: {}".format(accuracy_score(test_labels[:600], pred_labels_non_shuffled)))
print("SVM Accuracy with shuffled data: {}\n".format(accuracy_score(test_labels[:600], pred_labels)))

print("SVM Confusion matrix without shuffled data:\n{}".format(confusion_matrix(test_labels[:600], pred_labels_non_shuffled)))
print("SVM Classification report without shuffled data:\n{}\n\n".format(classification_report(test_labels[:600], pred_labels_non_shuffled)))

print("SVM Confusion matrix with shuffled data:\n{}".format(confusion_matrix(test_labels[:600], pred_labels)))
print("SVM Classification report with shuffled data:\n{}\n\n".format(classification_report(test_labels[:600], pred_labels)))

Chosing SVM params without shuffled data:  {'C': 1.273090909090909, 'kernel': 'rbf'} 

Chosing SVM params with shuffled data:  {'C': 0.001, 'kernel': 'rbf'} 

SVM Accuracy without shuffled data: 0.855
SVM Accuracy with shuffled data: 0.12166666666666667

SVM Confusion matrix without shuffled data:
[[52  0  0  0  0  1  0  0  0  0]
 [ 0 73  0  0  0  0  0  0  0  0]
 [ 1  1 57  0  1  0  0  3  1  0]
 [ 0  0  2 43  0 12  0  2  2  1]
 [ 0  0  1  0 54  0  2  0  0 10]
 [ 1  1  1  1  1 47  0  3  0  1]
 [ 2  0  5  0  1  2 42  0  0  0]
 [ 0  1  0  0  2  0  0 52  0  2]
 [ 2  0  3  2  1  2  1  1 37  3]
 [ 0  0  1  1  0  1  0  3  2 56]]
SVM Classification report without shuffled data:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94        53
           1       0.96      1.00      0.98        73
           2       0.81      0.89      0.85        64
           3       0.91      0.69      0.79        62
           4       0.90      0.81      0.85       

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Na podstawie powyższych rezultatów można zauważyc, że klasyfikator SVM działa lepiej dla obrazów, które nie zostały zmieszane. 

*   Accuracy dla obrazów wymiesznaych: 0.1217
*   Accuracy dla obrazów niewymieszanych: 0.855


**Las losowy**

In [10]:
parameters_for_cross_validation = {'max_depth':[5, 10, 15], 'n_estimators': [80,100,120], 'max_features': [30, 45, 60]}

non_shuffle_model = GridSearchCV(RandomForestClassifier(), parameters_for_cross_validation, cv=5, scoring='accuracy')
non_shuffle_model.fit(train_raw[:600], train_labels[:600])
non_shuffle_params = non_shuffle_model.best_params_
print("Chosing RandomForestClassifier params without shuffled data: ", non_shuffle_params, '\n')


shuffle_model = GridSearchCV(RandomForestClassifier(), parameters_for_cross_validation, cv=5, scoring='accuracy')
shuffle_model.fit(train_shuffle_data[:600], train_labels[:600])
params = shuffle_model.best_params_
print("Chosing RandomForestClassifier params with shuffled data: ", params, '\n')

pred_labels_non_shuffled = non_shuffle_model.predict(test_raw[:600])
pred_labels = shuffle_model.predict(test_shuffle_data[:600])

print("RFC Accuracy without shuffled data: {}".format(accuracy_score(test_labels[:600], pred_labels_non_shuffled)))
print("RFC Accuracy with shuffled data: {}\n".format(accuracy_score(test_labels[:600], pred_labels)))

print("RFC Confusion matrix without shuffled data:\n{}".format(confusion_matrix(test_labels[:600], pred_labels_non_shuffled)))
print("RFC Classification report without shuffled data:\n{}\n\n".format(classification_report(test_labels[:600], pred_labels_non_shuffled)))

print("RFC Confusion matrix with shuffled data:\n{}".format(confusion_matrix(test_labels[:600], pred_labels)))
print("RFC Classification report with shuffled data:\n{}\n\n".format(classification_report(test_labels[:600], pred_labels)))

Chosing RandomForestClassifier params without shuffled data:  {'max_depth': 10, 'max_features': 60, 'n_estimators': 120} 

Chosing RandomForestClassifier params with shuffled data:  {'max_depth': 5, 'max_features': 45, 'n_estimators': 100} 

RFC Accuracy without shuffled data: 0.82
RFC Accuracy with shuffled data: 0.115

RFC Confusion matrix without shuffled data:
[[51  0  0  0  0  1  1  0  0  0]
 [ 0 73  0  0  0  0  0  0  0  0]
 [ 0  2 55  1  0  1  0  4  1  0]
 [ 0  1  2 43  0  9  2  2  1  2]
 [ 1  2  1  0 52  0  2  0  0  9]
 [ 2  1  1  6  4 39  1  1  0  1]
 [ 2  0  3  0  4  2 41  0  0  0]
 [ 0  1  2  0  2  0  0 46  0  6]
 [ 2  0  2  2  0  1  1  0 35  9]
 [ 0  1  1  2  1  0  0  1  1 57]]
RFC Classification report without shuffled data:
              precision    recall  f1-score   support

           0       0.88      0.96      0.92        53
           1       0.90      1.00      0.95        73
           2       0.82      0.86      0.84        64
           3       0.80      0.69   

Na podstawie powyższych rezultatów można zauważyc, że klasyfikator RFC działa lepiej dla obrazów, które nie zostały zmieszane. 

*   Accuracy dla obrazów wymiesznaych: 0.115
*   Accuracy dla obrazów niewymieszanych: 0.82



**Sieć neuronowa**

In [13]:
encoded_train_labels = to_categorical(train_labels)
encoded_test_labels = to_categorical(test_labels)

parameters_for_cross_validation = {'epochs':[1,6,12], 'batch_size':[32,64,128]}
non_shuffle_network = KerasClassifier(create_model)
non_shuffle_model = GridSearchCV(estimator=non_shuffle_network, param_grid=parameters_for_cross_validation, cv=5)
non_shuffle_model.fit(train_raw[:600], encoded_train_labels[:600])
non_shuffle_params = non_shuffle_model.best_params_
print("Chosing Neural Network params with HOG data: ", non_shuffle_params)

shuffle_network = KerasClassifier(build_fn=create_model)
shuffle_model = GridSearchCV(estimator=shuffle_network, param_grid=parameters_for_cross_validation, cv=5)
shuffle_model.fit(train_shuffle_data[:600], encoded_train_labels[:600])
shuffle_params = shuffle_model.best_params_
print("Chosing Neural Network params without HOG data: ", shuffle_params)


pred_labels_non_shuffled = non_shuffle_model.predict(test_raw[:600])
pred_labels = shuffle_model.predict(test_shuffle_data[:600])

print("NN Accuracy without shuffled data: {}".format(accuracy_score(test_labels[:600], pred_labels_non_shuffled)))
print("NN Accuracy with shuffled data: {}\n".format(accuracy_score(test_labels[:600], pred_labels)))

print("NN Confusion matrix without shuffled data:\n{}".format(confusion_matrix(test_labels[:600], pred_labels_non_shuffled)))
print("NN Classification report without shuffled data:\n{}\n\n".format(classification_report(test_labels[:600], pred_labels_non_shuffled)))

print("NN Confusion matrix with shuffled data:\n{}".format(confusion_matrix(test_labels[:600], pred_labels)))
print("NN Classification report with shuffled data:\n{}\n\n".format(classification_report(test_labels[:600], pred_labels)))

4/4 [==============================] - 0s 5ms/step - loss: 38.3126 - accuracy: 0.6000
Epoch 1/6
15/15 [==============================] - 1s 8ms/step - loss: 153.0049 - accuracy: 0.3324
Epoch 2/6
15/15 [==============================] - 0s 7ms/step - loss: 4.9072 - accuracy: 0.8924
Epoch 3/6
15/15 [==============================] - 0s 7ms/step - loss: 5.8228 - accuracy: 0.8927
Epoch 4/6
15/15 [==============================] - 0s 7ms/step - loss: 3.7661 - accuracy: 0.9290
Epoch 5/6
15/15 [==============================] - 0s 7ms/step - loss: 2.9211 - accuracy: 0.9367
Epoch 6/6
4/4 [==============================] - 0s 5ms/step - loss: 17.1080 - accuracy: 0.8500
Epoch 1/6
15/15 [==============================] - 1s 7ms/step - loss: 127.9191 - accuracy: 0.3794
Epoch 2/6
15/15 [==============================] - 0s 9ms/step - loss: 10.3281 - accuracy: 0.8285
Epoch 3/6
15/15 [==============================] - 0s 8ms/step - loss: 4.5601 - accuracy: 0.8957
Epoch 4/6
15/15 [====================

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


NN Accuracy without shuffled data: 0.815
NN Accuracy with shuffled data: 0.11333333333333333

NN Confusion matrix without shuffled data:
[[42  0  1  0  0  1  0  6  3  0]
 [ 0 73  0  0  0  0  0  0  0  0]
 [ 0  0 59  0  0  1  1  2  1  0]
 [ 0  0  4 45  0 12  0  0  1  0]
 [ 0  2  0  1 53  1  3  0  0  7]
 [ 0  0  0  5  5 41  0  0  5  0]
 [ 1  0  1  1  1  3 43  2  0  0]
 [ 0  1  3  3  2  0  0 43  0  5]
 [ 0  2  3  0  2  2  2  1 39  1]
 [ 0  0  1  2  4  1  0  4  1 51]]
NN Classification report without shuffled data:
              precision    recall  f1-score   support

           0       0.98      0.79      0.88        53
           1       0.94      1.00      0.97        73
           2       0.82      0.92      0.87        64
           3       0.79      0.73      0.76        62
           4       0.79      0.79      0.79        67
           5       0.66      0.73      0.69        56
           6       0.88      0.83      0.85        52
           7       0.74      0.75      0.75        

Na podstawie powyższych rezultatów można zauważyc, że Sieć neuronowa działa lepiej dla obrazów, które nie zostały zmieszane. 

*   Accuracy dla obrazów wymiesznaych: 0.113
*   Accuracy dla obrazów niewymieszanych: 0.815



Dla wszystkich klasyfikatorów wyniki z surowymi, wymieszanymi danymi były o wiele niższe w stosunku do danych, które nie zostały wymieszane. 